# POS-Tagging
На этом семинаре мы рассмотрим некоторые библиотеки, позволяющие делать морфологический анализ, и даже обучим свой теггер на основе [BERT](https://habr.com/ru/post/436878/) (stay tuned!)

# NLTK
Разнообразные теггеры находятся в модуле `nltk.tag`. Можно скачивать дополнительные модели (вспомним, как это делается?)

In [5]:
!pip install flair

     |████████████████████████████████| 184kB 3.5MB/s 
     |████████████████████████████████| 655kB 53.7MB/s 
     |████████████████████████████████| 1.0MB 45.9MB/s 
     |████████████████████████████████| 798kB 20.1MB/s 
     |████████████████████████████████| 778kB 25.4MB/s 
     |████████████████████████████████| 184kB 53.1MB/s 
     |████████████████████████████████| 1.0MB 44.2MB/s 
     |████████████████████████████████| 348kB 49.4MB/s 
     |████████████████████████████████| 860kB 45.0MB/s 
  Created wheel for regex: filename=regex-2019.8.19-cp36-cp36m-linux_x86_64.whl size=609250 sha256=cd0213f45270613f2cf2f548b55fc95be2b72f4b277e65401c8c693b9db74a7c
  Stored in directory: /root/.cache/pip/wheels/90/04/07/b5010fb816721eb3d6dd64ed5cc8111ca23f97fdab8619b5be
  Created wheel for langdetect: filename=langdetect-1.0.7-cp36-none-any.whl size=993460 sha256=781a908b55b0c383c26780a418e5fb7691ec5338fee18e6dba66d16c1e5f4a1c
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7e

In [0]:
import nltk

In [7]:
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [9]:
brown.tagged_sents()

[[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS'), ('any', 'DTI'), ('irregularities', 'NNS'), ('took', 'VBD'), ('place', 'NN'), ('.', '.')], [('The', 'AT'), ('jury', 'NN'), ('further', 'RBR'), ('said', 'VBD'), ('in', 'IN'), ('term-end', 'NN'), ('presentments', 'NNS'), ('that', 'CS'), ('the', 'AT'), ('City', 'NN-TL'), ('Executive', 'JJ-TL'), ('Committee', 'NN-TL'), (',', ','), ('which', 'WDT'), ('had', 'HVD'), ('over-all', 'JJ'), ('charge', 'NN'), ('of', 'IN'), ('the', 'AT'), ('election', 'NN'), (',', ','), ('``', '``'), ('deserves', 'VBZ'), ('the', 'AT'), ('praise', 'NN'), ('and', 'CC'), ('thanks', 'NNS'), ('of', 'IN'), ('the', 'AT'), ('City', 'NN-TL'), ('of', 'IN-TL'), ('Atlant

In [0]:
# корпуса делятся на части - categories
print(brown.categories())

In [0]:
import nltk.tag
from nltk.corpus import brown

In [10]:
brown.tagged_words()

[('The', 'AT'), ('Fulton', 'NP-TL'), ...]

In [11]:
tags = [tag for (word, tag) in brown.tagged_words(categories='news')]
nltk.FreqDist(tags).max()

'NN'

In [12]:
default_tagger = nltk.tag.DefaultTagger('NN')
sentence = brown.sents()[5]
print(sentence)
default_tagger.tag(sentence)

['It', 'recommended', 'that', 'Fulton', 'legislators', 'act', '``', 'to', 'have', 'these', 'laws', 'studied', 'and', 'revised', 'to', 'the', 'end', 'of', 'modernizing', 'and', 'improving', 'them', "''", '.']


[('It', 'NN'),
 ('recommended', 'NN'),
 ('that', 'NN'),
 ('Fulton', 'NN'),
 ('legislators', 'NN'),
 ('act', 'NN'),
 ('``', 'NN'),
 ('to', 'NN'),
 ('have', 'NN'),
 ('these', 'NN'),
 ('laws', 'NN'),
 ('studied', 'NN'),
 ('and', 'NN'),
 ('revised', 'NN'),
 ('to', 'NN'),
 ('the', 'NN'),
 ('end', 'NN'),
 ('of', 'NN'),
 ('modernizing', 'NN'),
 ('and', 'NN'),
 ('improving', 'NN'),
 ('them', 'NN'),
 ("''", 'NN'),
 ('.', 'NN')]

Не очень здорово, верно? Давайте хранить самые частые слова и приписывать им их самый частый тег (а остальным пока приписывать `NN`)

In [13]:
fd = nltk.FreqDist(brown.words(categories='news'))
cfd = nltk.ConditionalFreqDist(brown.tagged_words(categories='news'))
most_freq_words = fd.most_common(1000)
likely_tags = dict((word, cfd[word].max()) for (word, _) in most_freq_words)
unigram_tagger = nltk.UnigramTagger(model=likely_tags, backoff=default_tagger)
unigram_tagger.tag(sentence)

[('It', 'PPS'),
 ('recommended', 'NN'),
 ('that', 'CS'),
 ('Fulton', 'NP-TL'),
 ('legislators', 'NN'),
 ('act', 'NN'),
 ('``', '``'),
 ('to', 'TO'),
 ('have', 'HV'),
 ('these', 'DTS'),
 ('laws', 'NNS'),
 ('studied', 'NN'),
 ('and', 'CC'),
 ('revised', 'NN'),
 ('to', 'TO'),
 ('the', 'AT'),
 ('end', 'NN'),
 ('of', 'IN'),
 ('modernizing', 'NN'),
 ('and', 'CC'),
 ('improving', 'NN'),
 ('them', 'PPO'),
 ("''", "''"),
 ('.', '.')]

In [14]:
# Можно легко оценить на каких-нибудь предложениях из того же корпуса
unigram_tagger.evaluate(brown.tagged_sents(categories='reviews'))

0.68452731918239

In [0]:
# Можно просто приписывать самый частый тег всем словам из корпуса
unigram_tagger = nltk.UnigramTagger(brown.tagged_sents(categories='news'))

## Задание
Используя [документацию](http://www.nltk.org/api/nltk.tag.html#nltk.tag.sequential.BigramTagger) и функцию `help()` обучите биграммный теггер. Сравните его с униграммным на примерах, где должно стать лучше (выбор тега зависит от контекста), например: `They wind back the clock.` vs. `The wind rises.`

In [0]:
#bigram_tagger = nltk.tag.sequential.BigramTagger(train = brown.tagged_sents(), 
#                                                 model=likely_tags, 
#                                                 backoff = default_tagger)

train = list(brown.tagged_sents())

Помимо этого в NLTK реализовано множество теггеров:
* regexp tagger
* ngram tagger
* Brill tagger
* CRF tagger
* etc.

## spaCy
[SpaCy](https://spacy.io/) — библиотека для обработки текстов, содержащая разные модули (токенайзер, pos-tagging, NER etc.)
Разработана специально для построения пайплайнов, сейчас нас будет интересовать токенизация и морфологический анализ.
Недостаток — некоторые модели (например, для NER) нужно загружать отдельно.

In [0]:
#!pip install spacy

In [0]:
#!python -m spacy download en_core_web_sm

In [23]:
nltk.download('webtext')
from nltk.corpus import webtext

[nltk_data] Downloading package webtext to /root/nltk_data...
[nltk_data]   Unzipping corpora/webtext.zip.


In [0]:
first_text_id = webtext.fileids()[0]
sentences = webtext.raw(first_text_id).split('\r\n')
text = " ".join(sentences[:10])

In [25]:
print(text)

Cookie Manager: "Don't allow sites that set removed cookies to set future cookies" should stay checked When in full screen mode Pressing Ctrl-N should open a new browser when only download dialog is left open add icons to context menu So called "tab bar" should be made a proper toolbar or given the ability collapse / expand. [XUL] Implement Cocoa-style toolbar customization. #ifdefs for MOZ_PHOENIX customize dialog's toolbar has small icons when small icons is not checked nightly builds and tinderboxen for Phoenix finish tearing prefs UI to pieces and then make it not suck


In [26]:
import spacy

# Загружаем весь пайплайн для английского
nlp = spacy.load("en_core_web_sm")

# Обрабатываем текст
doc = nlp(text)

# Выведем токены, леммы и теги
for i, s in enumerate(doc.sents):
    print("\n-- Sentence %d --" % i)
    for t in s:
        print(t.text, t.lemma_, t.pos_, sep="\t")


-- Sentence 0 --
Cookie	Cookie	PROPN
Manager	Manager	PROPN
:	:	PUNCT
"	"	PUNCT
Do	do	VERB
n't	not	ADV
allow	allow	VERB
sites	site	NOUN
that	that	DET
set	set	VERB
removed	remove	VERB
cookies	cookie	NOUN
to	to	PART
set	set	VERB
future	future	ADJ
cookies	cookie	NOUN
"	"	PUNCT
should	should	VERB
stay	stay	VERB
checked	check	VERB

-- Sentence 1 --
When	when	ADV
in	in	ADP
full	full	ADJ
screen	screen	NOUN
mode	mode	NOUN

-- Sentence 2 --
Pressing	press	VERB
Ctrl	Ctrl	PROPN
-	-	PUNCT
N	N	PROPN
should	should	VERB
open	open	VERB
a	a	DET
new	new	ADJ
browser	browser	NOUN
when	when	ADV
only	only	ADJ
download	download	NOUN
dialog	dialog	NOUN
is	be	VERB
left	leave	VERB
open	open	ADJ
add	add	NOUN
icons	icon	NOUN
to	to	ADP
context	context	NOUN
menu	menu	NOUN

-- Sentence 3 --
So	so	ADV
called	call	VERB
"	"	PUNCT
tab	tab	NOUN
bar	bar	NOUN

-- Sentence 4 --
"	"	PUNCT
should	should	VERB
be	be	VERB
made	make	VERB
a	a	DET
proper	proper	ADJ
toolbar	toolbar	NOUN
or	or	CCONJ
given	give	VERB
the	the	DET
abilit

## Самостоятельно:
возьмите любой текст на английском языке (например, скопируйте абзац из Википедии), примените к нему пайплайн spaCy, 
выведите все биграммы, состоящие из прилагательного и существительного.

**Бонус-трек**: перепишите ваш код в виде функции, которая принимает на вход текст и последовательность тегов, а возвращает энграммы, соответствующие данной последовательности. Протестируйте функцию на 3/4-граммах.

In [0]:
text = """
"""
### YOUR CODE HERE ###

# Flair
[Flair](https://github.com/zalandoresearch/flair) — библиотека для работы с векторными представлениями слов, содержит компоненты для решения 
разных задач NLP (sequence tagging, classification ...). Есть как готовые модели, так и возможность обучения на своих данных, дообучение предобученные векторных представлений и т.д.

Сегодня рассмотрим некоторые возможности flair для морфологического анализа на примере англоязычных данных.

In [0]:
# !pip install flair

In [27]:
# импортируем простейшие объекты
from flair.data import Sentence
from flair.models import SequenceTagger

# создаем предложение
sentence = Sentence('Moscow is the capital of Russia .')

# загружаем модель теггинга
tagger = SequenceTagger.load('pos')

# обрабатываем предложение
# ВАЖНО: объект `sentence` при этом меняется
tagger.predict(sentence)

print(sentence)
print(sentence.to_tagged_string())

2019-10-04 14:51:33,031 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.4/POS-ontonotes--h256-l1-b32-p3-0.5-%2Bglove%2Bnews-forward%2Bnews-backward-normal-locked0.5-word0.05--v0.4_0/en-pos-ontonotes-v0.4.pt not found in cache, downloading to /tmp/tmpzt_9czau


100%|██████████| 432218302/432218302 [00:19<00:00, 22517323.62B/s]

2019-10-04 14:51:52,745 copying /tmp/tmpzt_9czau to cache at /root/.flair/models/en-pos-ontonotes-v0.4.pt


2019-10-04 14:51:53,846 removing temp file /tmp/tmpzt_9czau
2019-10-04 14:51:53,907 loading file /root/.flair/models/en-pos-ontonotes-v0.4.pt
Sentence: "Moscow is the capital of Russia ." - 7 Tokens
Moscow <PROPN> is <VERB> the <DET> capital <NOUN> of <ADP> Russia <PROPN> . <PUNCT>


In [28]:
# если интересно узнать подробнее об этих объектах
help(sentence)

Help on Sentence in module flair.data object:

class Sentence(DataPoint)
 |  A Sentence is a list of Tokens and is used to represent a sentence or text fragment.
 |  
 |  Method resolution order:
 |      Sentence
 |      DataPoint
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __copy__(self)
 |  
 |  __getitem__(self, idx:int) -> flair.data.Token
 |  
 |  __init__(self, text:str=None, use_tokenizer:bool=False, labels:Union[List[flair.data.Label], List[str]]=None, language_code:str=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |  
 |  __len__(self) -> int
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __str__(self) -> str
 |      Return str(self).
 |  
 |  add_label(self, label:Union[flair.data.Label, str])
 |  
 |  add_labels(self, labels:Union[List[flair.data.Label], List[str]])
 |  
 |  add_token(self, token:Union[flair.data.Token, str])
 |  
 |  clear_embeddings(self, embedding_names:List[str]=None

У flair есть встроенные датасеты (`flair.datasets`), можно попробовать использовать их.

In [29]:
import flair.datasets
corpus = flair.datasets.UD_ENGLISH()

2019-10-04 14:57:20,885 https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-dev.conllu not found in cache, downloading to /tmp/tmpz4esi45i


1668174B [00:00, 37222916.73B/s]         

2019-10-04 14:57:20,991 copying /tmp/tmpz4esi45i to cache at /root/.flair/datasets/ud_english/en_ewt-ud-dev.conllu
2019-10-04 14:57:20,996 removing temp file /tmp/tmpz4esi45i


2019-10-04 14:57:21,320 https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-test.conllu not found in cache, downloading to /tmp/tmp17q79_io


1661985B [00:00, 34881733.83B/s]         

2019-10-04 14:57:21,430 copying /tmp/tmp17q79_io to cache at /root/.flair/datasets/ud_english/en_ewt-ud-test.conllu
2019-10-04 14:57:21,443 removing temp file /tmp/tmp17q79_io


2019-10-04 14:57:22,196 https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-train.conllu not found in cache, downloading to /tmp/tmp_cxdr_7y


13303045B [00:00, 78753281.36B/s]

2019-10-04 14:57:22,437 copying /tmp/tmp_cxdr_7y to cache at /root/.flair/datasets/ud_english/en_ewt-ud-train.conllu


2019-10-04 14:57:22,464 removing temp file /tmp/tmp_cxdr_7y
2019-10-04 14:57:22,900 Reading data from /root/.flair/datasets/ud_english
2019-10-04 14:57:22,901 Train: /root/.flair/datasets/ud_english/en_ewt-ud-train.conllu
2019-10-04 14:57:22,902 Test: /root/.flair/datasets/ud_english/en_ewt-ud-test.conllu
2019-10-04 14:57:22,903 Dev: /root/.flair/datasets/ud_english/en_ewt-ud-dev.conllu


In [30]:
corpus.test[0].to_tagged_string('upos')

'What <PRON> if <SCONJ> Google <PROPN> Morphed <VERB> Into <ADP> GoogleOS <PROPN> ? <PUNCT>'

In [0]:
type(corpus.test[0])

In [0]:
from random import choice
n = choice(range(len(corpus.test)))
sentence = corpus.test[n]
print(sentence.to_tagged_string('upos'))
print()

# помним, что теперь `sentence` поменяется
tagger.predict(sentence)

print(sentence)
print(sentence.to_tagged_string('upos'))

И наконец: **flair** позволяет обучать свои модели. Попробуем использовать эмбеддинги от Google, чтобы обучить свой теггер, вот [так](https://colab.research.google.com/drive/1OZN14wo1QGiwSpFhYuKPOPbzmJ2CzeWU).